In [17]:
#!import config/Settings.cs 

// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.23.230906.2-preview"
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;
using Microsoft.SemanticKernel.Planning.Stepwise;
using System.Text.Json;
using System.Collections.Generic;

//Create Kernel builder
var builder = new KernelBuilder();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

IKernel kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 0.23.230906.2-preview

In [18]:
#!import skills/Bank/ContaCorrente.cs 
#!import skills/Bank/Boleto.cs 
#!import skills/Bank/BankSkills.cs 


In [19]:
var contaCorrentPrincipal = new ContaCorrente("123", "eu@contoso.com");
contaCorrentPrincipal.Depositar(1000);

Dictionary<string, ContaCorrente> contasCorrentes = new Dictionary<string, ContaCorrente>() {
    { "André", new ContaCorrente("456", "andre@contoso.com") },
    { "João", new ContaCorrente("789", "joao@contoso.com") }
};


Boleto[] boletos = { Boleto.Criar("1", 450), Boleto.Criar("2", 150) , Boleto.Criar("3", 1500) };

var bankSkills = kernel.ImportSkill(new BankSkills(contaCorrentPrincipal, boletos, contasCorrentes), "BankSkills");

var planner = new StepwisePlanner(kernel);

In [20]:
// Create a plan for the ask
var ask = "Pagar todos os boletos e tranferir 300 para o João. Não deixe o saldo negativo, e retorne o saldo final e os boletos que foram pagos. Priorize os de maior valor";
var plan = planner.CreatePlan(ask);

//Console.WriteLine("Plan:");
//Console.WriteLine(plan.ToJson(true));
// Execute the plan
var result = await plan.InvokeAsync();



var steps = JsonSerializer.Deserialize<List<SystemStep>>(result.Variables["stepsTaken"]);
Console.WriteLine("Steps taken:");
foreach (SystemStep step in steps)
{
    Console.WriteLine("---------");
    Console.WriteLine("OriginalResponse : " + step.OriginalResponse );
    Console.WriteLine("Thought: " + step.Thought);
    Console.WriteLine("Action: " + step.Action);
    Console.WriteLine("Observation: " + step.Observation);
    Console.WriteLine("FinalAnswer: " + step.FinalAnswer);
}

Console.WriteLine("Resposta:");
Console.WriteLine(result.Result);


Boletos Pendentes: 
Numero:1 - Valor:450
Numero:2 - Valor:150
Numero:3 - Valor:1500

Saldo: 1000
Pagar Boleto 1 chamado
Saldo: 550
Pagar Boleto 2 chamado
Saldo: 400
ObterContaRelacionada João chamado
Transferir 300 para 789 chamado
Saldo: 100
Steps taken:
---------
OriginalResponse : [THOUGHT]
Para resolver essa questão, precisamos seguir os seguintes passos:

1. Obter os boletos pendentes.
2. Classificar os boletos em ordem decrescente de valor.
3. Pagar os boletos um por um, verificando o saldo após cada pagamento para garantir que não fique negativo.
4. Obter o número da conta do João.
5. Transferir 300 para o João, se o saldo permitir.
6. Obter o saldo final.

Vamos começar obtendo os boletos pendentes.[ACTION]
{
  "action": "BankSkills.BoletosPendentes"
}
Thought: Para resolver essa questão, precisamos seguir os seguintes passos:

1. Obter os boletos pendentes.
2. Classificar os boletos em ordem decrescente de valor.
3. Pagar os boletos um por um, verificando o saldo após cada pag